In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input

C:\Users\crsel\AppData\Local\Temp\ipykernel_23308\1412109789.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('open_gym.csv')

In [3]:
df.head()

,open_gym_start,open_gym_end,open_gym_activity,group,facility_title,location,total_females,total_males,total_residents,total_non_residents,total
0,2011-01-02 09:00:00,2011-01-02 11:30:00,Badminton,Everyone,Bond Park Community Center,Sycamore Gymnasium,0,3,2,1,3
1,2011-01-02 10:00:00,2011-01-02 12:00:00,Volleyball,Adult,Bond Park Community Center,Magnolia Gymnasium,0,2,1,1,2
2,2011-01-03 18:00:00,2011-01-03 22:00:00,Volleyball,Adult,Bond Park Community Center,Sycamore Gymnasium,0,2,1,1,2
3,2011-01-04 12:00:00,2011-01-04 14:00:00,Basketball,Adult,Bond Park Community Center,Sycamore Gymnasium,0,1,1,0,1
4,2011-01-04 18:00:00,2011-01-04 21:30:00,Open Gym,Everyone,Herbert C. Young Community Center,Coach Kay Yow Court,1,1,2,0,2


In [4]:
df.sort_values(['facility_title', 'open_gym_start'], inplace=True)
df.to_csv('open_gym_sorted.csv', index=False)
df = pd.read_csv('open_gym_sorted.csv', parse_dates=['open_gym_start', 'open_gym_end'])
year = df['open_gym_start'].dt.year
df['year'] = year
df['month'] = df['open_gym_start'].dt.month
df['day'] = df['open_gym_start'].dt.day
df['hour'] = df['open_gym_start'].dt.hour
df['weekday'] = df['open_gym_start'].dt.weekday
df['weekday_name'] = df['open_gym_start'].dt.day_name()
df['duration'] = df['open_gym_end'] - df['open_gym_start']
df['duration'] = df['duration'].dt.total_seconds() / 3600
df['duration'] = df['duration']
df.head()

,open_gym_start,open_gym_end,open_gym_activity,group,facility_title,location,total_females,total_males,total_residents,total_non_residents,total,year,month,day,hour,weekday,weekday_name,duration
0,2011-01-02 09:00:00,2011-01-02 11:30:00,Badminton,Everyone,Bond Park Community Center,Sycamore Gymnasium,0,3,2,1,3,2011,1,2,9,6,Sunday,2.5
1,2011-01-02 10:00:00,2011-01-02 12:00:00,Volleyball,Adult,Bond Park Community Center,Magnolia Gymnasium,0,2,1,1,2,2011,1,2,10,6,Sunday,2.0
2,2011-01-03 18:00:00,2011-01-03 22:00:00,Volleyball,Adult,Bond Park Community Center,Sycamore Gymnasium,0,2,1,1,2,2011,1,3,18,0,Monday,4.0
3,2011-01-04 12:00:00,2011-01-04 14:00:00,Basketball,Adult,Bond Park Community Center,Sycamore Gymnasium,0,1,1,0,1,2011,1,4,12,1,Tuesday,2.0
4,2011-01-05 15:00:00,2011-01-05 17:30:00,Basketball,Youth/Teen,Bond Park Community Center,Magnolia Gymnasium,0,1,0,1,1,2011,1,5,15,2,Wednesday,2.5


df['open_gym]

In [5]:
app = Dash(__name__)

months = ['All Months - Select A Month','Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
yaxis_title = 'Total People'
xtitle = ''

app.layout = html.Div([
    html.H1('Community Center Data', style={'width':'%50', 'display':'inline', 'backgroundColor': 'white', 'padding': '25px'}),
    html.Div([
        
        dcc.Dropdown(
            id='month-dropdown',
            options=[{'label': month, 'value': index} for index, month in enumerate(months)],
            value=0  
        ),
        dcc.Dropdown(
            id='year-dropdown',
            options=[
                {'label': 'All Years - Select Year', 'value': 0}
            ] + [{'label': year, 'value': year} for year in sorted(df['year'].unique())],
            value=0  
        ),
        dcc.Dropdown(
            id='xaxis-category-dropdown',
            options=[
                {'label': 'Scatter: Open Gym Activity', 'value': 'open_gym_activity'},
                {'label': 'Bar: Facility', 'value': 'facility_title'},
                {'label': 'Bar: Day', 'value': 'weekday_name'},
                {'label': 'Violin: Location', 'value': 'location'}
            ],
            value='open_gym_activity'
        ),
        dcc.Checklist(
            id='yaxis-category-checklist',
            options=[
                {'label': 'Total People', 'value': 'total'},
                {'label': 'Total Residents', 'value': 'total_residents'},
                {'label': 'Non Residents', 'value': 'total_non_residents'},
                {'label': 'Total Males', 'value': 'total_males'},
                {'label': 'Total Females', 'value': 'total_females'}

            ],
            value=['total_residents'],
            labelStyle={'display': 'block'}
        ),
    ], style={'width': '50%', 'backgroundColor': 'white'}),
    dcc.Graph(id='graph-content', config={'displayModeBar': False}) 
])

@callback(
    Output('graph-content', 'figure'),
    Input('month-dropdown', 'value'),
    Input('year-dropdown', 'value'),
    Input('xaxis-category-dropdown', 'value'),
    Input('yaxis-category-checklist', 'value')
)
def update_graph(selected_month, selected_year, xaxis_category, yaxis_category):
    df_filtered = df.copy()
    if selected_month > 0:
        df_filtered = df_filtered[df_filtered['month'] == selected_month]
    if selected_year > 0:
        df_filtered = df_filtered[df_filtered['year'] == selected_year]

    # Prepare the figure with conditional titles based on selection
    titles = f'Total People vs. {xaxis_category.replace("_", " ").title()}'
    if selected_year > 0:
        titles += f' in {selected_year}'
    else:
        titles += ' Across All Years'
    if selected_month > 0:
        titles += f' in {months[selected_month]}'
    else:
        titles += ' for All Months'

    figure = go.Figure()

    if yaxis_category != []:         
        if xaxis_category == 'open_gym_activity':
            X = df_filtered[xaxis_category].unique()
            if 'total' in yaxis_category:
                figure.add_trace(go.Scatter(name='Total People', y= df_filtered[xaxis_category],x=df_filtered['total'], mode='markers',hovertext=df_filtered['weekday_name'], marker=dict(color='black', size = 25, line=dict(color='black', width=1))))
            if 'total_residents' in yaxis_category:
                figure.add_trace(go.Scatter(name='Total Residents', x= df_filtered['total_residents'],y=df_filtered[xaxis_category], mode='markers', hovertext=df_filtered['weekday_name'], marker=dict(color='lightgreen', size = 20, line=dict(color='green', width=1))))
            if 'total_non_residents' in yaxis_category:
                figure.add_trace(go.Scatter(name='Total Non Residents', x=df_filtered['total_non_residents'], y=df_filtered[xaxis_category], mode='markers', hovertext=df_filtered['weekday_name'], marker=dict(color='darkred',size = 15, line=dict(color='red', width=1))))
            if 'total_males' in yaxis_category:
                figure.add_trace(go.Scatter(name='Total_Males', x=df_filtered['total_males'], y=df_filtered[xaxis_category], mode='markers', hovertext=df_filtered['weekday_name'], marker=dict(color='lightblue', size = 10, line=dict(color='blue', width=1))))
            if 'total_females' in yaxis_category:
                figure.add_trace(go.Scatter(name='Total Females', x=df_filtered['total_females'],  y=df_filtered[xaxis_category], mode='markers', hovertext=df_filtered['weekday_name'],  marker=dict(color='pink', size = 6, line=dict(color='purple', width=1))))
        elif xaxis_category == 'facility_title':
            if 'total' in yaxis_category:
                figure.add_trace(go.Bar(name='Total People', x=df_filtered[xaxis_category], y=df_filtered['total'], hovertext=df_filtered['weekday_name'], marker=dict(color='black', line=dict(color='black', width=1))))
            if 'total_residents' in yaxis_category:
                figure.add_trace(go.Bar(name='Total Residents', x=df_filtered[xaxis_category], y=df_filtered['total_residents'], hovertext=df_filtered['weekday_name'], marker=dict(color='green', line = dict(color='green', width=1))))
            if 'total_non_residents' in yaxis_category:
                figure.add_trace(go.Bar(name='Total Non Residents', x=df_filtered[xaxis_category], y=df_filtered['total_non_residents'], hovertext=df_filtered['weekday_name'], marker=dict(color='red', line = dict(color='red', width=1))))
            if 'total_males' in yaxis_category:
                figure.add_trace(go.Bar(name='Total_Males', x=df_filtered[xaxis_category], y=df_filtered['total_males'], hovertext=df_filtered['weekday_name'], marker=dict(color='blue', line = dict(color='blue', width=1))))
            if 'total_females' in yaxis_category:
                figure.add_trace(go.Bar(name='Total Females', x=df_filtered[xaxis_category], y=df_filtered['total_females'], hovertext=df_filtered['weekday_name'], marker=dict(color='pink', line=dict(color='pink', width=1))))
        elif xaxis_category == 'weekday_name':
            df_filtered = df_filtered.reset_index()
            dfweekdays = df_filtered[['total','total_females', 'total_males', 'total_residents', 'total_non_residents', 'weekday_name']].copy()
            dfweekdays = dfweekdays.groupby('weekday_name').sum().reset_index()
            if 'total' in yaxis_category:
                figure.add_trace(go.Bar(name='Total People', x=dfweekdays[xaxis_category], y=dfweekdays['total'], marker=dict(color='black')))
            if 'total_residents' in yaxis_category:
                figure.add_trace(go.Bar(name='Total Residents', x=dfweekdays[xaxis_category], y=dfweekdays['total_residents'],marker=dict(color='green')))
            if 'total_non_residents' in yaxis_category:
                figure.add_trace(go.Bar(name='Total Non Residents', x=dfweekdays[xaxis_category], y=dfweekdays['total_non_residents'],marker=dict(color='red')))
            if 'total_males' in yaxis_category:
                figure.add_trace(go.Bar(name='Total_Males', x=dfweekdays[xaxis_category], y=dfweekdays['total_males'],marker=dict(color='blue')))
            if 'total_females' in yaxis_category:
                figure.add_trace(go.Bar(name='Total Females', x=dfweekdays[xaxis_category], y=dfweekdays['total_females'], marker=dict(color='pink')))
        elif xaxis_category == 'location':
            if 'total' in yaxis_category:
                figure.add_trace(go.Violin(name='Total People', x=df_filtered[xaxis_category], y=df_filtered['total'], pointpos = 2,hovertext=df_filtered['weekday_name'],  marker=dict(color='black', line=dict(color='black', width=0.3))))
            if 'total_residents' in yaxis_category:
                figure.add_trace(go.Violin(name='Total Residents', x=df_filtered[xaxis_category], y=df_filtered['total_residents'],hovertext=df_filtered['weekday_name'], pointpos = -2,marker=dict(color='green', line=dict(color='black', width=0.3))))
            if 'total_non_residents' in yaxis_category:
                figure.add_trace(go.Violin(name='Total Non Residents', x=df_filtered[xaxis_category], y=df_filtered['total_non_residents'],hovertext=df_filtered['weekday_name'], pointpos = -1,marker=dict(color='red',line=dict(color='black', width=0.3))))
            if 'total_males' in yaxis_category:
                figure.add_trace(go.Violin(name='Total_Males', x=df_filtered[xaxis_category], y=df_filtered['total_males'], pointpos =0, hovertext=df_filtered['weekday_name'],marker=dict(color='blue', line=dict(color='black', width=0.3))))
            if 'total_females' in yaxis_category:
                figure.add_trace(go.Violin(name='Total Females', x=df_filtered[xaxis_category], y=df_filtered['total_females'], pointpos = 1,hovertext=df_filtered['weekday_name'], marker=dict(color='pink', line=dict(color='black', width=0.3))))
            
        if xaxis_category == 'open_gym_activity':
            yaxis_title = 'Activity'
        else:
            yaxis_title = 'Total People'
    else:
        yaxis_title = 'No Values Selected'
        figure = px.scatter(
            df_filtered,
            x=[0],
            y=[0],
            title=titles,
        )
    if xaxis_category == 'open_gym_activity':
        xtitle = 'Total People'
    else:
        xtitle = xaxis_category.replace('_', ' ').title()

    figure.update_layout(
        xaxis_title=xtitle,
        yaxis_title=yaxis_title,
        font = {
            'size': 20,  # You can adjust the size as needed
            'color': 'black',  # You can specify the color
            'family': "Arial, sans-serif",  # You can adjust the font family as needed
        },
        title={
        'text': titles,
        'y':0.95,  # Adjusts the vertical position to be at the top
        'x':0.95,
        'font': {
            'size': 25,  # You can adjust the size as needed
            'color': 'black',  # You can specify the color
            'family': "Arial, sans-serif",  # You can adjust the font family as needed
        }
        },

        height=600
    )

    figure.layout.yaxis.fixedrange = True
    figure.layout.xaxis.fixedrange = True
    return figure

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)